# TensorBoard

Source: https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html

В этом ноутбуке мы научимся:
- Запускать TensorBoard
- Делать записи в виде изображений и скаляров
- Исследовать архитектуру моделей

In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [2]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'hymenoptera_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### 1. TensorBoard setup 

Now we’ll set up TensorBoard, importing tensorboard from torch.utils and defining a SummaryWriter, our key object for writing information to TensorBoard.

In [4]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs')
# Load the TensorBoard notebook extension
%load_ext tensorboard

### 2. Writing to TensorBoard 
Now let’s write an image to our TensorBoard - specifically, a grid - using make_grid.

In [5]:
# get some random training images
dataiter = iter(dataloaders['train'])
images, labels = dataiter.next()

# create grid of images
img_grid = torchvision.utils.make_grid(images, normalize=True)

# write to tensorboard
writer.add_image('Train images', img_grid)

In [6]:
%tensorboard --logdir='runs'

Reusing TensorBoard on port 6006 (pid 6095), started 1:39:16 ago. (Use '!kill 6095' to kill it.)

## 3. Inspect the model using TensorBoard 

One of TensorBoard’s strengths is its ability to visualize complex model structures. Let’s visualize the model we built.

In [3]:
net = models.alexnet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [7]:
writer.add_graph(net, images)
writer.close()